## Functions

__Function for extracting number from the string__

In [88]:
def num_extractor(string):
    
    import re
    
    # Regex expression for finding the number
    s = [float(s) for s in re.findall(r'-?\d+\.?\d*', string)]
    
    num = s
    
    if len(s) == 2:
    
        num = s[0]*1000 + s[1]
        
    if type(num) == list:
        
        num = float(num[0])
        
    return num

__Splitting the price in high and low__

In [89]:
def price_split(string):
    
    # Splitting algorithm
    split_string = string.split('-')
    len_split = len(split_string)
    
    
    low_price = num_extractor(split_string[0])
    high_price = num_extractor(split_string[0])
    
    # Vehicle has low and high price
    if len_split == 2:
        
        low_price = num_extractor(split_string[0])
        high_price = num_extractor(split_string[1])
        
    # Conversion of lakhs to numbers
    
    if low_price/10 < 1:
        
        low_price = low_price*1e5
        
    if high_price/10 < 1:
        
        high_price = high_price*1e5
    
        
    return (low_price,high_price)


__Specification Transposing Funstion__

In [90]:
def spec_transpose(df):
    
    # Column name
    spec_name = df['spec_columns'].values[0]
    
    # Renaming the specification value column
    df.rename(columns = {'spec_values':spec_name},inplace=True)
    df.reset_index(drop=True, inplace=True)
    df = df[['product_name',spec_name]]
    
    return df

## Data Import

In [91]:
import os

In [92]:
os.chdir('G:\Sanit_Horizontal\Data Analysis\web_scraping-main\Bikedekho\Data Files')

In [93]:
df = pd.read_csv('all_spec_extractor.csv', header = 0, encoding= 'unicode_escape')
df.drop(columns = [df.columns[0],df.columns[1]], axis = 1, inplace = True)

## Data Cleaning

In [94]:
spec_col = df.drop(columns = df.columns[-1], axis = 1)
spec_col.dropna(axis = 0,inplace = True)
spec_col.reset_index(inplace = True,drop = True)

In [95]:
spec_val = df.drop(columns = df.columns[-2], axis = 1)
spec_val.dropna(axis = 0,inplace = True)
spec_val.reset_index(inplace = True,drop = True)

In [96]:
all_spec = pd.concat([spec_col,spec_val['spec_values']],axis = 1)

In [97]:
all_spec.head()

,product_name,price,spec_columns,spec_values
0,Oben Rorr,Rs.1.03 Lakh*,Motor Power (w),1000
1,Oben Rorr,Rs.1.03 Lakh*,Starting,Push Button Start
2,Oben Rorr,Rs.1.03 Lakh*,Bluetooth Connectivity,"Bluetooth,WiFi"
3,Oben Rorr,Rs.1.03 Lakh*,Anti Theft Alarm,Yes
4,Oben Rorr,Rs.1.03 Lakh*,Speedometer,Digital


## Price Column Conversion

In [98]:
price_list = list(all_spec['price'])

In [99]:
low_list = []
high_list = []

In [100]:
for string in price_list:
    
#     print(string)
    low, high = price_split(string)
    
    low_list.append(low)
    high_list.append(high)   

In [101]:
all_spec = pd.concat([all_spec,pd.DataFrame({'price_low':low_list,'price_high':high_list})],axis = 1)

In [102]:
all_spec.head()

,product_name,price,spec_columns,spec_values,price_low,price_high
0,Oben Rorr,Rs.1.03 Lakh*,Motor Power (w),1000,103000.0,103000.0
1,Oben Rorr,Rs.1.03 Lakh*,Starting,Push Button Start,103000.0,103000.0
2,Oben Rorr,Rs.1.03 Lakh*,Bluetooth Connectivity,"Bluetooth,WiFi",103000.0,103000.0
3,Oben Rorr,Rs.1.03 Lakh*,Anti Theft Alarm,Yes,103000.0,103000.0
4,Oben Rorr,Rs.1.03 Lakh*,Speedometer,Digital,103000.0,103000.0


## Separting the brand list

In [103]:
brand = all_spec.drop(columns = ['spec_columns','spec_values'],axis = 1)
brand.drop_duplicates(inplace = True)
brand.reset_index(drop = True,inplace=True)

In [104]:
brand.head()

,product_name,price,price_low,price_high
0,Oben Rorr,Rs.1.03 Lakh*,103000.0,103000.0
1,Tork Kratos,Rs.1.22 - 1.37 Lakh*,122000.0,137000.0
2,Ampere Magnus EX,"Rs.81,900*",81900.0,81900.0
3,Bounce Infinity E1,"Rs.47,499 - 96,799*",47499.0,96799.0
4,Hero Electric Optima,"Rs.67,190 - 85,190*",67190.0,85190.0


In [105]:
spec_list = sorted(list(all_spec['spec_columns'].unique()))

## Spec to column conversion

__Transposing the all_spec DataFrame__

In [106]:
concat_df = brand.copy()

In [107]:
for spec in spec_list:
    
    dum = all_spec[all_spec['spec_columns']== spec] 
    concat_df = pd.merge(concat_df,spec_transpose(dum), how='outer')

C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_14828\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'spec_values':spec_name},inplace=True)
C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_14828\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'spec_values':spec_name},inplace=True)
C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_14828\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_14828\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'spec_values':spec_name},inplace=True)
C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_14828\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'spec_values':spec_name},inplace=True)
C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_14828\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_14828\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'spec_values':spec_name},inplace=True)
C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_14828\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'spec_values':spec_name},inplace=True)
C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_14828\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_14828\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'spec_values':spec_name},inplace=True)
C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_14828\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns = {'spec_values':spec_name},inplace=True)
C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_14828\3400413341.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_gui

In [108]:
concat_df.drop_duplicates(inplace=True)
concat_df.reset_index(drop=True,inplace=True)

In [109]:
list(concat_df.keys())

['product_name',
 'price',
 'price_low',
 'price_high',
 '0-100 Kmph (sec)',
 '0-40 Kmph (sec)',
 'ABS',
 'ARAI Mileage',
 'Acceleration (0-100 Kmph)',
 'Additional Features',
 'Additional Features Of Variant',
 'Additional Storage',
 'Adjustable Windscreen',
 'Anti Theft Alarm',
 'Artificial Exhaust Sound System',
 'Battery Capacity',
 'Battery Type',
 'Battery Warranty',
 'Bluetooth Connectivity',
 'Body Graphics',
 'Body Type',
 'Boot Space',
 'Brakes Front',
 'Brakes Rear',
 'Braking (60-0 Kmph)',
 'Braking Type',
 'Call/SMS Alerts',
 'Carry Hook',
 'Carry hook',
 'Central Locking',
 'Charger Output',
 'Charging At Charging Station',
 'Charging At Home',
 'Charging Network / Battery Swapping Network',
 'Charging Point',
 'Charging Station Locater',
 'Charging Time(0-100%)',
 'Charging Time(0-80%)',
 'Claimed Range',
 'Clock',
 'Continious Power',
 'Continuous Power',
 'Cooling System',
 'Cruise Control',
 'DRLs',
 'Display',
 'Distance to Empty Indicator',
 'Drive Type',
 'Dry Weig

## Extracting battery and motor related data

In [110]:
req_keys = []

In [111]:
for string in spec_list:
    
    if any([x in string for x in matches]):
        
        print(string)
        req_keys.append(string)

Battery Capacity
Battery Type
Battery Warranty
Charging Network / Battery Swapping Network
Low Battery Indicator
Motor IP Rating
Motor Power (w)
Motor Type
Motor Warranty
Removable Battery Weight
Swappable Battery
Torque (Motor)


In [112]:
battery_motor_dataframe = concat_df[['product_name','price_low','price_high']+req_keys+['Drive Type']]

In [113]:
battery_motor_dataframe.head()

,product_name,price_low,price_high,Battery Capacity,Battery Type,Battery Warranty,Charging Network / Battery Swapping Network,Low Battery Indicator,Motor IP Rating,Motor Power (w),Motor Type,Motor Warranty,Removable Battery Weight,Swappable Battery,Torque (Motor),Drive Type
0,Oben Rorr,103000.0,103000.0,4.4 kwh,NaN,3 Years,NaN,Yes,NaN,1000,IPMSM Motor,3 Years,NaN,NaN,NaN,Belt Drive
1,Tork Kratos,122000.0,137000.0,4 kWh,NaN,NaN,Yes,Yes,IP67,9000,PMAC,NaN,NaN,NaN,38 Nm,Mid Drive Electric Motor
2,Ampere Magnus EX,81900.0,81900.0,60 V / 38.25 Ah,Lithium Ion,NaN,NaN,NaN,NaN,2100,BLDC,NaN,NaN,NaN,NaN,Hub Motor
3,Bounce Infinity E1,47499.0,96799.0,1.9 KWh,Lithium Ion,NaN,NaN,Yes,NaN,1500,BLDC,NaN,NaN,Yes,85 Nm,Hub Motor
4,Hero Electric Optima,67190.0,85190.0,51.2 V / 30 Ah,NaN,NaN,NaN,NaN,NaN,1200,NaN,NaN,NaN,NaN,NaN,Hub Motor


## Swapable Battery Vehicles

In [114]:
battery_motor_dataframe.keys()

Index(['product_name', 'price_low', 'price_high', 'Battery Capacity',
       'Battery Type', 'Battery Warranty',
       'Charging Network / Battery Swapping Network', 'Low Battery Indicator',
       'Motor IP Rating', 'Motor Power (w)', 'Motor Type', 'Motor Warranty',
       'Removable Battery Weight', 'Swappable Battery', 'Torque (Motor)',
       'Drive Type'],
      dtype='object')

In [115]:
swappable_bat_vehicles = battery_motor_dataframe[~battery_motor_dataframe['Swappable Battery'].isnull()]
swappable_bat_vehicles.sort_values('price_high',inplace=True)

C:\Users\spbhatkar.AHEAD\AppData\Local\Temp\ipykernel_14828\2505045082.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  swappable_bat_vehicles.sort_values('price_high',inplace=True)


In [116]:
swappable_bat_vehicles = swappable_bat_vehicles[['product_name','price_low','price_low','Battery Capacity','Battery Type','Motor Power (w)','Motor Type','Torque (Motor)','Drive Type']]
swappable_bat_vehicles.reset_index(drop = True, inplace=True)

In [117]:
swappable_bat_vehicles

,product_name,price_low,price_low,Battery Capacity,Battery Type,Motor Power (w),Motor Type,Torque (Motor),Drive Type
0,Raftaar Galaxy,51900.0,51900.0,64 V / 30 Ah,LFP,250,BLDC,NaN,Hub Motor
1,EeVe Ahava,62499.0,62499.0,60 V / 27 Ah,Lead Acid,250,BLDC,NaN,Hub Motor
2,Benling Kriti,64151.0,64151.0,48 V / 28 Ah,Lithium-ion,250,Brushless Motor,NaN,Hub Motor
3,Benling Falcon,69540.0,69540.0,60 V / 30 Ah,Lithium-ion,250,BLDC,NaN,Hub Motor
4,Okaya Freedum,74900.0,74900.0,1.4 kWh,Lithium Ion,250,BLDC,NaN,Hub Motor
5,Jitendra JET 320,73480.0,73480.0,1.56 Kwh,Lithium Ion,250,BLDC,NaN,Hub Motor
6,Gemopai Ryder SuperMax,79999.0,79999.0,60 V / 30 Ah,Lithium Ion,2700,BLDC,NaN,Hub Motor
7,iVOOMi Eco,71999.0,71999.0,60 V / 35 Ah,Lithium Ion,250,NaN,NaN,Hub Motor
8,Gemopai Ryder,60000.0,60000.0,48 V / 40 Ah,Lithium-ion,250,Brushless Motor,NaN,NaN
9,Jitendra JMT 1000 3K,84417.0,84417.0,3.120 Kwh,Lithium Ion,1000,BLDC,NaN,Hub Motor


## Export to csv

In [118]:
concat_df.to_csv('all_vehicle_spec_data.csv')

In [ ]:
Branch: Main
Commit: C4